In [1]:
!pip install ultralytics 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.4 MB/s eta 0:00:00


In [3]:
import yaml

# Cấu hình đường dẫn dataset
# Lưu ý: YOLO cần đường dẫn tuyệt đối tới folder 'images'
data_config = {
    'path': '/kaggle/input/scut-head/PartA', # Root path (optional nhưng tốt để define context)
    'train': '/kaggle/input/scut-head/PartA/train/images',
    'val':   '/kaggle/input/scut-head/PartA/valid/images',
    'test':  '/kaggle/input/scut-head/PartA/test/images',
    
    # Class configuration
    'nc': 1,           # Số lượng class: chỉ có 1 class là "head"
    'names': ['head']  # Tên class
}

# Lưu file yaml
yaml_path = '/kaggle/working/scut_head.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_config, f)

print(f"✅ Đã tạo file config tại: {yaml_path}")
# %cat {yaml_path} # Uncomment để xem nội dung file


✅ Đã tạo file config tại: /kaggle/working/scut_head.yaml


In [4]:
# 2. Tạo file YAML cho YOLO26-P2 (chính thức từ Ultralytics)
p2_yaml_content = """
# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license
# Ultralytics YOLO26 object detection model with P2/4 - P5/32 outputs
# Model docs: https://docs.ultralytics.com/models/yolo26
# Task docs: https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 80  # number of classes (sẽ override sau)
end2end: True
reg_max: 1
depth_multiple: 1.00
width_multiple: 1.50
max_channels: 512

backbone:
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 2, C3k2, [256, False, 0.25]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 2, C3k2, [512, False, 0.25]]
  - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
  - [-1, 2, C3k2, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 2, C3k2, [1024, True]]
  - [-1, 1, SPPF, [1024, 5, 3, True]] # 9
  - [-1, 2, C2PSA, [1024]] # 10

head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 2, C3k2, [512, True]] # 13
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 2, C3k2, [256, True]] # 16 (P3/8-small)
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 2], 1, Concat, [1]] # cat backbone P2
  - [-1, 2, C3k2, [128, True]] # 19 (P2/4-xsmall)
  - [-1, 1, Conv, [128, 3, 2]]
  - [[-1, 16], 1, Concat, [1]] # cat head P3
  - [-1, 2, C3k2, [256, True]] # 22 (P3/8-small)
  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 13], 1, Concat, [1]] # cat head P4
  - [-1, 2, C3k2, [512, True]] # 25 (P4/16-medium)
  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 10], 1, Concat, [1]] # cat head P5
  - [-1, 1, C3k2, [1024, True, 0.5, True]] # 28 (P5/32-large)
  - [[19, 22, 25, 28], 1, Detect, [nc]] # Detect(P2, P3, P4, P5)
"""

p2_yaml_path = '/kaggle/working/yolo26x-p2.yaml'
with open(p2_yaml_path, 'w') as f:
    f.write(p2_yaml_content.strip())

print(f"✅ Đã tạo file YOLO26-P2 YAML tại: {p2_yaml_path}")

✅ Đã tạo file YOLO26-P2 YAML tại: /kaggle/working/yolo26x-p2.yaml


In [5]:
# 3. Train với P2 head
from ultralytics import YOLO

print("🚀 Load pretrained YOLO26x + override với P2 architecture...")
model = YOLO('yolo26x.pt')  # pretrained weights
model = YOLO(p2_yaml_path).load('yolo26x.pt')  # override arch P2 + map weights (backbone khớp)

model.info()

print("🚀 Bắt đầu Training với P2 head...")
results = model.train(
    data='/kaggle/working/scut_head.yaml',
    epochs=100,
    imgsz=800,                  # Có thể tăng lên 800-896 nếu VRAM cho phép để tận dụng P2
    batch=4,                    # GIẢM xuống 8 hoặc 4 để tránh OOM (P2 + x nặng hơn base)
    patience=20,                # Tăng lên 20 cho an toàn
    optimizer='AdamW',
    lr0=0.001,
    cos_lr=True,
    # Augmentation mạnh hơn cho small/occluded heads
    mosaic=1.0,
    close_mosaic=10,
    mixup=0.2,                  # Tăng nhẹ
    copy_paste=0.4,             # Tăng cho occlusion
    fliplr=0.5,
    scale=0.6,
    rect=False,                   
    project='SCUT_Head_Project',
    name='yolo26x_p2_v0_800',
    exist_ok=True,
    verbose=True,
    amp=True,                   # Mixed precision để tiết kiệm VRAM
    freeze=10,
    single_cls=True,
)

print("✅ Training hoàn tất!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Load pretrained YOLO26x + override với P2 architecture...
Transferred 553/1370 items from pretrained weights
YOLO26x-p2 summary: 489 layers, 106,143,360 parameters, 106,143,360 gradients, 296.9 GFLOPs
🚀 Bắt đầu Training với P2 head...
Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.4, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/scut_head.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dn